In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
import time
import numpy as np
import torch
from torch import nn, optim, autograd
from math import pi
from sklearn.model_selection import RepeatedKFold
from bayes_opt import BayesianOptimization
from sklearn.metrics import r2_score
import torch.nn.functional as F


torch.manual_seed(123456)
np.random.seed(123456)


def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0.0)


class Unit3(nn.Module):
    def __init__(self, in_N, out_N,actf):
        super(Unit3, self).__init__()
        self.in_N = in_N
        self.out_N = out_N
        self.actf = actf
        self.L = nn.Linear(in_N, out_N)

    def forward(self, x):
        actf=self.actf
        x1 = self.L(x)
        if actf==0:
            x2 = torch.tanh(x1)
        elif actf==1:
            x2 = torch.sigmoid(x1) 
        elif actf==2:
            x2 = torch.relu(x1)
        elif actf==3:
            x2 = torch.selu(x1)
        elif actf==4:
            x2 = F.softmax(x1, dim=1)
        return x2
    
class NN3(nn.Module):
    def __init__(self, in_N, width1, depth1,width2, depth2,out_N,bn,dp,dprate,actf):
        super(NN3, self).__init__()
        self.width1 = width1
        self.width2 = width2
        self.depth1 = depth1
        self.depth2 = depth2
        self.bn = bn
        self.dp = dp
        self.dprate = dprate
        self.actf = actf
        self.in_N = in_N
        self.out_N = out_N
        self.stack = nn.ModuleList()
        self.stack.append(Unit3(in_N, width1[0],actf))
        if bn==1:
            self.stack.append(nn.BatchNorm1d(width1[0]))
        for i in range(1,depth1):
            self.stack.append(Unit3(width1[i-1], width1[i],actf))
        
        if dp==1:
            self.stack.append(nn.Dropout(p=dprate))
        if depth2==1:
            self.stack.append(Unit3(width1[i], width2[0],1)) 
        else:
            self.stack.append(Unit3(width1[i], width2[0],actf))    
            for i in range(1,depth2-1):
                self.stack.append(Unit3(width2[i-1], width2[i],actf))
            self.stack.append(Unit3(width2[depth2-2], width2[depth2-1],4)) 
            
    def forward(self, x):
        for i in range(len(self.stack)):
            x = self.stack[i](x)
        return x


def nn_cl_bo2(activation,dropout, dropout_rate,normalization,batch_size,layers1, layers2,neurons,learning_rate): #Low-fedility model:
 
    
    epoch = 1500
    kfold=10
    
    layers1 = round(layers1)
    layers2 = round(layers2)
    neurons = round(neurons)
    batch_size = round(batch_size) 
    activation = round(activation)
    bnorm=0
    if normalization>0.5:
        bnorm=1
    dout=0
    if dropout>0.5:
        dout=1
    load=0
    PATH="checkpoint/model-9.pt"
    if load==1:
        checkpoint = torch.load(PATH)
        model_h.load_state_dict(checkpoint['model_h_state_dict'])
        optimizer2.load_state_dict(checkpoint['optimizer2_state_dict'])
    cv = RepeatedKFold(n_splits=kfold, n_repeats=1, random_state=1)  
    Tot_acc=np.empty([epoch,3*kfold]) 
    Tot_acc[:]=np.nan
    cv_acc=[]
    cv_num=0
    
    post_analysis_train=torch.zeros([kfold,np.shape(xlo)[0],(16)]) 
    post_analysis_test=torch.zeros([kfold,np.shape(xlo_test)[0],(16)])
    
    for train_ix, test_ix in cv.split(xlo):
        post_analysis_train_1=torch.zeros([0,16])
        x_lo_train, x_lo_val = xlo[train_ix], xlo[test_ix]
        y_lo_train, y_lo_val = ylo[train_ix], ylo[test_ix]    
        if cv_num==0:
            with open('xlo_fold1.npy','wb') as f:
                np.save(f,x_lo_train)
            with open('ylo_fold1.npy','wb') as f:
                np.save(f,y_lo_train)
        L1=[neurons]*layers1
        L2=[neurons]*layers2+[8]
        model_h = NN3(35,L1,layers1,L2,layers2+1, 8,bnorm,dout,dropout_rate,activation)
        model_h.apply(weights_init)
        optimizer2 = optim.AdamW([{'params': model_h.parameters()}], lr=learning_rate) 
        num_batch=np.shape(x_lo_train)[0]//batch_size 
        loss2_value = 1
        earlystop=[]
        
        for it in range(epoch): 
            model_h.train()
            print("EPOCH=",it)
            for batch_i in range(num_batch-1):
                loss_fn = torch.nn.MSELoss()
                pred_h = model_h(torch.from_numpy(x_lo_train).float()[batch_i*batch_size:(batch_i+1)*batch_size,:])
                loss3=loss_fn(pred_h,torch.from_numpy(y_lo_train[batch_i*batch_size:(batch_i+1)*batch_size,:]).float())
                
                
                if it==epoch-1:
                    Conf_train=torch.cat(((pred_h).float(),torch.from_numpy(y_lo_train[batch_i*batch_size:(batch_i+1)*batch_size,:]).int()),dim=1)
                    post_analysis_train_1=torch.cat((post_analysis_train_1,Conf_train),dim=0)

                loss2=loss3
                loss2_value = loss2.item()
                optimizer2.zero_grad()
                loss2.backward()
                optimizer2.step()
            
            batch_i=num_batch-1 
            pred_h = model_h(torch.from_numpy(x_lo_train).float()[batch_i*batch_size:,:])
            loss3=loss_fn(pred_h,torch.from_numpy(y_lo_train[batch_i*batch_size:,:]).float())
            
            if it==epoch-1:
                Conf_train=torch.cat(((pred_h).float(),torch.from_numpy(y_lo_train[batch_i*batch_size:,:]).int()),dim=1)
                post_analysis_train_1=torch.cat((post_analysis_train_1,Conf_train),dim=0)
                post_analysis_train[cv_num,0:np.shape(post_analysis_train_1)[0],:]=post_analysis_train_1
      
            loss2=loss3
            loss2_value = loss2.item()
            optimizer2.zero_grad()
            loss2.backward()
            optimizer2.step()
            loss3=loss_fn(pred_h,torch.from_numpy(y_lo_train[batch_i*batch_size:,:]).float())
            acc_eval_train=loss3.detach().numpy()
            Tot_acc[it,3*cv_num]=acc_eval_train
            
            #Validate
            model_h.eval()
            pred_2h_star = model_h(torch.from_numpy(x_lo_val).float())
            loss3=loss_fn(pred_2h_star,torch.from_numpy(y_lo_val).float())
            acc_eval_val=loss3.detach().numpy()
            Tot_acc[it,3*cv_num+1]=acc_eval_val
            
            #Test
            pred_2h_star_test = model_h(torch.from_numpy(xlo_test).float())
            loss3=loss_fn(pred_2h_star_test,torch.from_numpy(ylo_test).float())
            
            acc_hi_0_test=(pred_2h_star_test).float()
            acc_hi_1_test=torch.from_numpy(ylo_test).float()
            
            if it==epoch-1:
                Conf_test=torch.cat((acc_hi_0_test,acc_hi_1_test),dim=1)
                post_analysis_test[cv_num,0:np.shape(Conf_test)[0],:]=Conf_test
        
            acc_eval_test=loss3.detach().numpy()
            Tot_acc[it,3*cv_num+2]=acc_eval_test
            
            earlystop=np.append(earlystop,acc_eval_val)
            if it==epoch-1:
                cv_acc=np.append(cv_acc,acc_eval_val) 
            
            if cv_num==0:
                PATH="checkpoint/model-"+str(it)+".pt"
                torch.save({
                        'model_h_state_dict': model_h.state_dict(),
                        'optimizer2_state_dict': optimizer2.state_dict(),
                        }, PATH)
                with open('CNN_Accuracy_fold1.npy', 'wb') as f:
                    np.save(f, Tot_acc)
                with open('Post_analysis_train_fold1.npy', 'wb') as f:
                    np.save(f, post_analysis_train.detach().numpy())
                with open('Post_analysis_test_fold1.npy', 'wb') as f:
                    np.save(f, post_analysis_test.detach().numpy())


        cv_num=cv_num+1
    
    with open('CNN_Accuracy.npy', 'wb') as f:
        np.save(f, Tot_acc)
    with open('Post_analysis_train.npy', 'wb') as f:
        np.save(f, post_analysis_train.detach().numpy())
    with open('Post_analysis_test.npy', 'wb') as f:
        np.save(f, post_analysis_test.detach().numpy())
        
    return np.mean(cv_acc)

start_time = time.time()

xlo=np.load('xlo.npy')
ylo=np.load('ylo.npy')
xlo_test=np.load('xlo_test.npy')
ylo_test=np.load('ylo_test.npy')
nn_cl_bo2(activation=0,dropout=1,dropout_rate=0.29791,normalization=1,batch_size=1000,layers1=10,layers2=1,neurons=86,learning_rate=0.00065)

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds") 